# This notebook is to test a single batch run in ADAM

In [1]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config('localdev')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])
aps = AdamProcessingService(auth_rest)

## Example Inputs

In [3]:
# Keplerian elements state vector
keplerian_elements = {
    'semi_major_axis_km': 448793612,
    'eccentricity': 0.1,
    'inclination_deg': 90,
    'ra_of_asc_node_deg': 91,
    'arg_of_pericenter_deg': 92,
    'true_anomaly_deg': 93,
    'gm': 132712440041.9394
}

# Uncertainties on elements
keplerian_sigma = {
    'semi_major_axis': 100,
    'eccentricity': 0.001,
    'inclination': 1,
    'ra_of_asc_node': 2,
    'arg_of_pericenter': 3,
    'true_anomaly': 4,
}

state_vec = [-150874809.2,
             -187234595.3,
             -73785026.7,
             14.64403935,
             -11.75744819,
             -5.583528281]

cartesian_sigma = {
    "x": 10,
    "y": 10,
    "z": 10,
    "x_dot": 1,
    "y_dot": 1,
    "z_dot": 1    
}


### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2001-01-02T01:13:46.62Z',  # propagation start time in ISO format
    'end_time': '2010-12-31T01:13:46.62Z',  # propagation end time in ISO format

    #'keplerianSigma': keplerian_sigma,
    'description': 'Jupyter Example',
    'cartesianSigma' : cartesian_sigma,
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    "stopOnImpactDistanceMeters": 500000,
    "closeApproachRadiusFromTargetMeters": 7000000000

})

opm_params = OpmParams({
    'epoch': '2001-01-02T01:13:46Z',
    # 'keplerian_elements': keplerian_elements,
    'state_vector': state_vec,
})


### Submit and Run Propagation

In [5]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, '0dc1e8b0-4f92-46ad-8838-c9e9eca6935c', '093a2424-9dfb-4cae-88bf-a9077659e8ca')
print(batch_run)

{'job_uuid': 'f57905a5-976c-4497-886a-b04c95a39cc3', 'results': None}


### Get Status

In [ ]:
print(batch_run.check_status())
# batch_run.wait_for_complete()
print()
print(batch_run.check_status())

CREATED


### Get Number of Runs

In [8]:
# Get ephemeris of specified part
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 11


In [9]:
### Get Summary Statistics

In [10]:
stats = batch_run.get_summary()
print(stats)

{'misses': 10, 'close_approach': 2, 'impacts': 1, 'total': 13, 'pc': 0.07692307692307693}


### Get Ephemeris for specific run

In [11]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

https://storage.googleapis.com/adam-beam-storage/output/job/efaa4ecc-c6d0-4d57-a769-cd400a3a0997/stk-ephemerides/MISS/run-9-00000-of-00005.e
stk.v.11.0
BEGIN Ephemeris
ScenarioEpoch 02 Jan 2001 01:13:46.620000
CentralBody SUN
CoordinateSystem ICRF
InterpolationMethod HERMITE
InterpolationOrder 5
NumberOfEphemerisPoints 3652

EphemerisTimePosVel
0.000000000000e+00 -1.508747971597e+11 -1.872346068199e+11 -7.378502618297e+10 1.323872360237e+04 -1.166633736818e+04 -6.604274171372e+03
8.640000000000e+04 -1.497262911171e+11 -1.882367452065e+11 -7.435333496361e+10 1.334687748682e+04 -1.153125174409e+04 -6.550977763176e+03
1.728000000000e+05 -1.485684889508e+11 -1.892271970070e+11 -7.491702757591e+10 1.345391259109e+04 -1.139581493084e+04 -6.497418691953e+03
2.592000000000e+05 -1.474014870808e+11 -1.902059319784e+11 -7.547608139412e+10 1.355983451445e+04 -1.126002922028e+04 -6.443598526127e+03
3.456000000000e+05 -1.462253814487e+11 -1.911729200713e+11 -7.603047392511e+10 1.366464872520e+04 -1.

### Get ending state vector

In [14]:
# Get the end state vector (uncomment to use)
end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
print(f'First close end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
print(len(end_state_vectors))
print(f'First impact end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.MISS)
print("Miss State Vectors")
print(len(end_state_vectors))
print(f'First miss end state: {end_state_vectors[0]}')


Close State Vectors
First close end state: [138335374118.59653, 51196673879.72533, 22195699580.749737]
Impact State Vectors
1
First impact end state: [775389611553.0438, 1652126811982.3506, -1143538336690.8086]
Miss State Vectors
10
First miss end state: [69224450.98563689, 79297887.22769228, 30371295.27024037]
